In [10]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [64]:
# replace only year to get info for desirable year
response = requests.get("https://www.baseball-reference.com/leagues/MLB/2017-schedule.shtml")

In [65]:
soup = BeautifulSoup(response.content, features= "html.parser")

In [66]:
txt = soup.find_all("p", attrs= {'class':"game"})

In [67]:
boxes_list = []

for e in txt:
    var = e.find("em")
    if var == None:
        continue
    else:
        boxes_list.append(var.find("a").get("href"))

In [68]:
len(boxes_list)

2468

In [69]:
def refinedInfo(lst,split_el):
    name = ['day','Start Time', 'Attendance', 'Venue', 'Game Duration', 'Game']
    dct = dict(zip(name,[0]*len(name)))
    for n in lst:
        t = n.split(split_el) 
        if 'Start Time' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'Attendance' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'Venue' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'Game Duration' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'day' in n:
            dct['day'] = n
        elif 'Game' in n:
            dct['Game'] = n
        
            
    return list(dct.values())

In [70]:
col_name = ['Team_1','Team_1_score','Team_2','Team_2_score','Day&Date','Start Time','Attendance', 'Venue', 'Game Duration', 'SegmentOfDay']
df = pd.DataFrame(columns= col_name)
for i,half_url in enumerate(boxes_list):
    url = "https://www.baseball-reference.com" + half_url
    response_1 = requests.get(url)
    soup_1 = BeautifulSoup(response_1.content, features= "html.parser")
    var_imd = soup_1.title.get_text().split(',')[:-2]
    score = [s.get_text().replace('\n','') for s in soup_1.find_all('div',attrs= {'class':"score"})]
    if score == []:
        score = ['NA', 'NA']
    
    if len(var_imd) >2:
        teams = var_imd[0].split(' vs. ')
        txt_1 = soup_1.find_all("div", attrs= {'class':"tabular has_groups"})
        imd = [j.get_text() for j in txt_1[0].find_all("div")]
        imd_1_ = imd[0].replace('\n','|').split('|||')[:-1]
        imd_2 = refinedInfo(imd_1_,'|')
    else:
        teams = var_imd[-1].split(' at ')
        txt_1 = soup_1.find_all("div", attrs= {'class':"scorebox_meta"})
        imd = [j.get_text() for j in txt_1[0].find_all("div")][:-1]
        imd_2 = refinedInfo(imd,': ')
   
    df.loc[i] = [teams[0]] + [score[0]] + [teams[1]] + [score[1]] + imd_2

In [71]:
df.shape

(2468, 10)

In [72]:
df

,Team_1,Team_1_score,Team_2,Team_2_score,Day&Date,Start Time,Attendance,Venue,Game Duration,SegmentOfDay
0,San Francisco Giants,5,Arizona Diamondbacks Box Score,6,"Sunday, April 2, 2017",1:10 p.m. Local,"49,016",Chase Field,3:23,"Day Game, on grass"
1,Chicago Cubs,3,St. Louis Cardinals Box Score,4,"Sunday, April 2, 2017",7:38 p.m. Local,"47,566",Busch Stadium III,3:33,"Night Game, on grass"
2,New York Yankees,3,Tampa Bay Rays Box Score,7,"Sunday, April 2, 2017",1:12 p.m. Local,"31,042",Tropicana Field,3:21,"Day Game, on turf"
3,Toronto Blue Jays,2,Baltimore Orioles Box Score,3,"Monday, April 3, 2017",3:09 p.m. Local,"45,667",Oriole Park at Camden Yards,3:41,"Day Game, on grass"
4,Pittsburgh Pirates,3,Boston Red Sox Box Score,5,"Monday, April 3, 2017",2:06 p.m. Local,"36,594",Fenway Park,3:03,"Day Game, on grass"
5,Philadelphia Phillies,4,Cincinnati Reds Box Score,3,"Monday, April 3, 2017",4:10 p.m. Local,"43,804",Great American Ball Park,3:04,"Day Game, on grass"
6,Seattle Mariners,0,Houston Astros Box Score,3,"Monday, April 3, 2017",7:15 p.m. Local,"41,678",Minute Maid Park,2:29,"Night Game, on grass"
7,San Diego Padres,3,Los Angeles Dodgers Box Score,14,"Monday, April 3, 2017",1:10 p.m. Local,"53,701",Dodger Stadium,2:52,"Day Game, on grass"
8,Colorado Rockies,7,Milwaukee Brewers Box Score,5,"Monday, April 3, 2017",1:11 p.m. Local,"43,336",Miller Park,3:26,"Day Game, on grass"
9,Kansas City Royals,1,Minnesota Twins Box Score,7,"Monday, April 3, 2017",3:16 p.m. Local,"39,615",Target Field,2:47,"Day Game, on grass"


In [73]:
df['Team_2'] = df['Team_2'].apply(lambda x: x.split(' Box ')[0])
df['Day&Date'] = df['Day&Date'].apply(lambda x: x.replace('||Date|',''))

In [74]:
df

,Team_1,Team_1_score,Team_2,Team_2_score,Day&Date,Start Time,Attendance,Venue,Game Duration,SegmentOfDay
0,San Francisco Giants,5,Arizona Diamondbacks,6,"Sunday, April 2, 2017",1:10 p.m. Local,"49,016",Chase Field,3:23,"Day Game, on grass"
1,Chicago Cubs,3,St. Louis Cardinals,4,"Sunday, April 2, 2017",7:38 p.m. Local,"47,566",Busch Stadium III,3:33,"Night Game, on grass"
2,New York Yankees,3,Tampa Bay Rays,7,"Sunday, April 2, 2017",1:12 p.m. Local,"31,042",Tropicana Field,3:21,"Day Game, on turf"
3,Toronto Blue Jays,2,Baltimore Orioles,3,"Monday, April 3, 2017",3:09 p.m. Local,"45,667",Oriole Park at Camden Yards,3:41,"Day Game, on grass"
4,Pittsburgh Pirates,3,Boston Red Sox,5,"Monday, April 3, 2017",2:06 p.m. Local,"36,594",Fenway Park,3:03,"Day Game, on grass"
5,Philadelphia Phillies,4,Cincinnati Reds,3,"Monday, April 3, 2017",4:10 p.m. Local,"43,804",Great American Ball Park,3:04,"Day Game, on grass"
6,Seattle Mariners,0,Houston Astros,3,"Monday, April 3, 2017",7:15 p.m. Local,"41,678",Minute Maid Park,2:29,"Night Game, on grass"
7,San Diego Padres,3,Los Angeles Dodgers,14,"Monday, April 3, 2017",1:10 p.m. Local,"53,701",Dodger Stadium,2:52,"Day Game, on grass"
8,Colorado Rockies,7,Milwaukee Brewers,5,"Monday, April 3, 2017",1:11 p.m. Local,"43,336",Miller Park,3:26,"Day Game, on grass"
9,Kansas City Royals,1,Minnesota Twins,7,"Monday, April 3, 2017",3:16 p.m. Local,"39,615",Target Field,2:47,"Day Game, on grass"


In [36]:
df.to_excel('MLB_2019_.xlsx')